In [7]:
import pandas as pd
import numpy as np
import re
import unicodedata

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=UserWarning)

In [8]:
#Carga de datos de IPM
df_IPM = pd.read_csv('data/Hogares_IPM_2018.csv', sep=";")
df_IPM.head()
df_IPM["ipm"] = df_IPM["ipm"].str.replace(",", ".").astype(float)
IPM_DEPARTAMENTOS = df_IPM.groupby('DEPARTAMENTO')['ipm'].mean().reset_index()
IPM_DEPARTAMENTOS

,DEPARTAMENTO,ipm
0,5,0.212018
1,8,0.230189
2,11,0.158780
3,13,0.265944
4,15,0.211427
5,17,0.207054
6,18,0.246883
7,19,0.229898
8,20,0.261580
9,23,0.264616


In [ ]:
# 1) Normalizar la tabla de IPM departamental
# df_ipm: columnas originales -> 'DEPARTAMENTO' (código depto) e 'ipm'
ipm_depto_df = IPM_DEPARTAMENTOS.rename(columns={'DEPARTAMENTO': 'dept_code', 'ipm': 'ipm_depto'}).copy()
ipm_depto_df['dept_code'] = pd.to_numeric(ipm_depto_df['dept_code'], errors='coerce').astype('Int64')

df_CNC = pd.read_excel('data/CNC_2018.xlsx')
# 2) Partir del listado de municipios
muni_df = (
    df_CNC[['PB']].dropna().drop_duplicates().copy()
)

# Extraer código de departamento desde PB (robusto a ceros a la izquierda)
muni_df['dept_code'] = (
    muni_df['PB'].astype(str)
                 .str.extract(r'(\d+)')[0]   # solo dígitos
                 .str.zfill(5)               # asegurar 5 dígitos (DDMMM)
                 .str[:2]                    # tomar 2 primeros
                 .astype('Int64')
)
muni_df.reset_index(drop=True, inplace=True)


df_CNC_IPM = muni_df.merge(ipm_depto_df, on='dept_code', how='inner')

display(df_CNC_IPM)

,PB,dept_code,ipm_depto
0,8001,8,0.230189
1,5001,5,0.212018
2,68001,68,0.207674
3,11001,11,0.158780
4,54001,54,0.264164
5,76001,76,0.194793
6,13001,13,0.265944
7,70001,70,0.277548
8,23079,23,0.264616
9,70708,70,0.277548


In [ ]:
# Cargar nuevo dataset de penetración de internet fijo y hacer merge con CNC + IPM
internet = pd.read_csv("data/Internet_Fijo_Penetración_Municipio_20250826.csv")


# ensure numeric
internet['AÑO'] = pd.to_numeric(internet['AÑO'], errors='coerce')
internet['TRIMESTRE'] = pd.to_numeric(internet['TRIMESTRE'], errors='coerce')

# filter: Año = 2018 and TRIMESTRE = 4
internet_2018q4 = internet.loc[(internet['AÑO'] == 2018) & (internet['TRIMESTRE'] == 4)].copy()


display(internet.columns)

# Unir con el dataset de encuestas agregadas
df_CNC_IPM_PI = df_CNC_IPM.merge(
    internet_2018q4[["COD_MUNICIPIO", "INDICE"]],
    left_on="PB", right_on="COD_MUNICIPIO", how="inner"
)

display(df_CNC_IPM_PI)

Index(['AÑO', 'TRIMESTRE', 'COD_DEPARTAMENTO', 'DEPARTAMENTO', 'COD_MUNICIPIO',
       'MUNICIPIO', 'No. ACCESOS FIJOS A INTERNET', 'POBLACIÓN DANE',
       'INDICE'],
      dtype='object')

,PB,dept_code,ipm_depto,COD_MUNICIPIO,INDICE
0,8001,8,0.230189,8001,"15,81"
1,5001,5,0.212018,5001,"24,42"
2,68001,68,0.207674,68001,"24,13"
3,11001,11,0.158780,11001,"25,31"
4,54001,54,0.264164,54001,"13,90"
5,76001,76,0.194793,76001,"21,38"
6,13001,13,0.265944,13001,"15,48"
7,70001,70,0.277548,70001,"13,30"
8,23079,23,0.264616,23079,"0,56"
9,70708,70,0.277548,70708,"1,96"


In [11]:

# Cargar datos de examen saber 11

saber = pd.read_csv("data/Examen_Saber_11_20181.txt", sep=";", usecols=["estu_cod_reside_mcpio","punt_global"], low_memory=False)

icfes_prom_muni = (
    saber.dropna(subset=["estu_cod_reside_mcpio", "punt_global"])
         .groupby("estu_cod_reside_mcpio", as_index=False)["punt_global"]
         .mean()
         .rename(columns={"punt_global": "saber_punt_global_mean"})
)



df_CNC_IPM_PI_SABER = df_CNC_IPM_PI.merge(
    icfes_prom_muni[["estu_cod_reside_mcpio", "saber_punt_global_mean"]],
    left_on="PB", right_on="estu_cod_reside_mcpio", how="inner"
)

df_maestro_global_variables_munnicipio = df_CNC_IPM_PI_SABER[["PB","dept_code","ipm_depto","INDICE","saber_punt_global_mean"]]

display(df_maestro_global_variables_munnicipio)

,PB,dept_code,ipm_depto,INDICE,saber_punt_global_mean
0,8001,8,0.230189,"15,81",286.622745
1,5001,5,0.212018,"24,42",256.242312
2,68001,68,0.207674,"24,13",302.117450
3,11001,11,0.158780,"25,31",288.624269
4,54001,54,0.264164,"13,90",258.778450
5,76001,76,0.194793,"21,38",291.885904
6,13001,13,0.265944,"15,48",252.610512
7,70001,70,0.277548,"13,30",247.569620
8,23079,23,0.264616,"0,56",244.000000
9,70708,70,0.277548,"1,96",237.125000


In [12]:
# Obtengo variables de interés del dataset CNC original

df_CNC_vars_interes = df_CNC[['PB','INDICADOR','NIVEL_PIRAMIDE','RANGO_EDAD','PB1','SEXO','ESTRATO','P33']].copy()
df_CNC_vars_interes.fillna({'P33': 0}, inplace=True)

#valido is hay nulos en las variables de interés
na_counts = df_CNC_vars_interes.isna().sum().sort_values(ascending=False)
na_pct = (df_CNC_vars_interes.isna().mean() * 100).round(2)
na_report = (
    pd.DataFrame({'n_nulls': na_counts, 'pct_nulls': na_pct})
      .sort_values('n_nulls', ascending=False)
)
display(na_report)



,n_nulls,pct_nulls
PB,0,0.0
INDICADOR,0,0.0
NIVEL_PIRAMIDE,0,0.0
RANGO_EDAD,0,0.0
PB1,0,0.0
SEXO,0,0.0
ESTRATO,0,0.0
P33,0,0.0


In [13]:
display(df_CNC_vars_interes)

df_modelo = df_CNC_vars_interes.merge(
    df_maestro_global_variables_munnicipio,
    on="PB", how="inner"
)

df_modelo = df_modelo.drop("dept_code", axis=1)
display(df_modelo)

,PB,INDICADOR,NIVEL_PIRAMIDE,RANGO_EDAD,PB1,SEXO,ESTRATO,P33
0,8001,0.000000,0,7,1,2,2,0.0
1,5001,0.000000,0,6,1,2,2,0.0
2,5001,0.000000,0,6,1,2,2,0.0
3,5001,0.076923,1,4,1,1,2,1.0
4,68001,0.461538,2,4,1,1,2,2.0
...,...,...,...,...,...,...,...,...
3478,81001,0.076923,1,2,2,2,1,4.0
3479,81001,0.000000,0,4,2,1,1,6.0
3480,81001,0.000000,0,2,2,1,1,6.0
3481,81001,0.230769,2,3,1,2,1,1.0


,PB,INDICADOR,NIVEL_PIRAMIDE,RANGO_EDAD,PB1,SEXO,ESTRATO,P33,ipm_depto,INDICE,saber_punt_global_mean
0,8001,0.000000,0,7,1,2,2,0.0,0.230189,"15,81",286.622745
1,5001,0.000000,0,6,1,2,2,0.0,0.212018,"24,42",256.242312
2,5001,0.000000,0,6,1,2,2,0.0,0.212018,"24,42",256.242312
3,5001,0.076923,1,4,1,1,2,1.0,0.212018,"24,42",256.242312
4,68001,0.461538,2,4,1,1,2,2.0,0.207674,"24,13",302.117450
...,...,...,...,...,...,...,...,...,...,...,...
3478,81001,0.076923,1,2,2,2,1,4.0,0.246648,"7,99",247.956044
3479,81001,0.000000,0,4,2,1,1,6.0,0.246648,"7,99",247.956044
3480,81001,0.000000,0,2,2,1,1,6.0,0.246648,"7,99",247.956044
3481,81001,0.230769,2,3,1,2,1,1.0,0.246648,"7,99",247.956044


In [14]:
# Export df_modelo as a CSV file
output_filename = 'df_modelo.csv'
df_modelo.to_csv(output_filename, index=False)

print(f"DataFrame df_modelo exported to {output_filename}")

DataFrame df_modelo exported to df_modelo.csv
